## Aquiring Data (0.1)

Using ravelry.com's api.  Limit of 100k per call.  As of Friday November 19, 665,465 knitting patterns. Api does not currently allow for sort-by searches directly.  Will need to segment and pull out in sequential chunks, then filter down after the fact. 

#### PLAN OF ATTACK
* get pattern categories list
* start with one category
* then iterate through grab patterns for each if <100k
* combine into one monster dataframe

* write a script to interate through all id's grab info, write to csv... append with all info

In [9]:
import pandas as pd
import numpy as np

import json
import requests
from requests.auth import HTTPBasicAuth
from pprint import pprint

from config import basic_auth_username, basic_auth_password
from config import basic_auth_username_read_only, basic_auth_password_read_only

### Try current user call (my account)

In [10]:
def search_current_user():
    """ returns current (my) user metadata """
    response =requests.get('https://api.ravelry.com/current_user.json', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
    if response.status_code != 200:
        print("Something's not quite right - please take a look, status_code = {}".format(response.status_code))
    else:
        return response.json()
    
search_current_user()

{'user': {'id': 264731,
  'username': 'upsidedown',
  'tiny_photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_tiny.JPG',
  'small_photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_small.JPG',
  'photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_large.JPG',
  'large_photo_url': 'https://avatars-d.ravelrycache.com/upsidedown/122038936/IMG_2763_xlarge.JPG'}}

In [460]:
# response =requests.get('https://api.ravelry.com/projects/DonnaYB/list.json?sort=rating_', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
# response.json()

### More general Search

In [138]:
# def search(params, endpoint):
#         """ gets data from ravelry api with given end point (string) and parameters (dictionary)
#         outlined in the api documentation """
#         base_url = 'https://api.ravelry.com/'
#         params = params
#         response =requests.get('https://api.ravelry.com/'+endpoint, params=params,auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
#         if response.status_code != 200:
#             print("Something's not quite right - please take a look, status_code = {}".format(response.status_code))
#         return(response.json())

#### get pattern categories

In [142]:
# # get categories
# response =requests.get('https://api.ravelry.com/pattern_categories/list.json')
# categories = response.json()

# # pprint(categories['pattern_categories']['children'])
# # ugh - this is too massive!

# # Ratings = 4 & knitting

In [157]:
# endpoint = 'patterns/search.json'
# params = {'query' : {'craft':'knitting'},"page": 1,"page_size": 500}# 'year published': '2020'
# # params = {'query' : {'craft':'knitting', 'sort by': 'best match'}, 'limit':500, "page": 1,"page_size": 100}
# response =requests.get('https://api.ravelry.com/'+end_point, params=params, auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
# # response =requests.get('https://api.ravelry.com/search.json?patterns/knitting', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))

## Grab actual patterns

In [498]:
# For knitting search:
page=2
response =requests.get('https://api.ravelry.com/patterns/search.json?craft=knitting&rating=4&sort=created_&page_size=500&page={}'.format(page),auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
search_result = response.json()

ids = []
for i in range(len(search_result['patterns'])):
    ids.append(search_result['patterns'][i]['id'])
    
id_list = [str(x) for x in ids]
spaced_ids = '+'.join(id_list)
spaced_ids

response =requests.get(f'https://api.ravelry.com/patterns.json?ids={spaced_ids}', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
patterns = response.json()
# patterns

In [355]:
# user
response =requests.get(f'https://api.ravelry.com/people/264731.json', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
users = response.json()
users

In [13]:
def get_search_results(page):
    response =requests.get(f'https://api.ravelry.com/patterns/search.json?craft=knitting&rating=4&sort=created_&page_size=500&page={page}',auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
    search_results = response.json()
    return search_results

def get_pattern_details(search_results):
    ids = []
    for i in range(len(search_result['patterns'])):
        ids.append(search_result['patterns'][i]['id'])

    id_list = [str(x) for x in ids]
    spaced_ids = '+'.join(id_list)
    spaced_ids

    response =requests.get(f'https://api.ravelry.com/patterns.json?ids={spaced_ids}', auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
    return response.json()

In [14]:
def get_all_patterns():
    try:
        for page in range(1,4):
            search_results = get_search_results(page)
            json_data = get_pattern_details(search_results)
            new_parsed_patterns = parse_json_response_into_df(json_data)
            
#             patterns_df = #append df

            print('yay parsed pattterns!')
            filter_and_save_csv(new_parsed_patterns)
            print('yay saved!')
    except Exception as e:
            print(e, 'Stopped on page {}'.format(page))
#     try:

#     except:
#         print('uh oh - saving failed')


get_all_patterns()

name 'search_result' is not defined Stopped on page 1


In [326]:
# {'paginator': {'last_page': 1000,
#                'page': 1,
#                'page_count': 1000,
#                'page_size': 100,
#                'results': 100000},d

### For Pattern Details
#### For a singular result:

In [414]:
# For one result:
response =requests.get('https://api.ravelry.com/patterns/573.json',auth=HTTPBasicAuth(basic_auth_username, basic_auth_password))
patterns = response.json()
# pprint(patterns)

#### For more than one result:

In [229]:
pattern = response.json()
page_count = pattern['paginator']['page_count']
last_page = pattern['paginator']['last_page']
# pprint(pattern['patterns'])

In [2]:
# Note this only works for multiple call 
# json_data = response.json()

def parse_json_response_into_df(json_data):
    
#     instantiate lists
    id_ = []
    name = []
    favorites_count = []
    projects_count = []
    difficulty_average = []
    difficulty_count = []
    rating_average = []
    rating_count = []
    pattern_type_id = []
    pattern_type_names = []
    pattern_type_clothing = []
    yarn_weight = []
    photos_url =[]
    craft_id = []
    yarn_weight = []
    pattern_needle_sizes = []
    yardage = []
    yarn_weight_description = []
    yardage_max = []
    yardage = []
    gauge = []
    gauge_divisor = []
    free = []
    downloadable = []
    queued_projects_count =[]
    attributes = []
    generally_available = []
    yarn_weight_name = []
    yardage_description =[]
    pattern_attributes = []
    categories = []
    name_permalink =[]

    # parse json response 
    for i in pattern['patterns']:
        id_.append(pattern['patterns'][i]['id'])
        name.append(pattern['patterns'][i]['name'])
        name_permalink.append(pattern['patterns'][i]['permalink'])
        favorites_count.append(pattern['patterns'][i]['favorites_count'])
        projects_count.append(pattern['patterns'][i]['projects_count'])
        difficulty_average.append(pattern['patterns'][i]['difficulty_average'])
        difficulty_count.append(pattern['patterns'][i]['difficulty_count'])
        queued_projects_count.append(pattern['patterns'][i]['queued_projects_count'])
        rating_average.append(pattern['patterns'][i]['rating_average'])
        rating_count.append(pattern['patterns'][i]['rating_count'])
        downloadable.append(pattern['patterns'][i]['downloadable'])
        free.append(pattern['patterns'][i]['free'])
        gauge_divisor.append(pattern['patterns'][i]['gauge_divisor'])
        gauge.append(pattern['patterns'][i]['gauge'])
        yardage.append(pattern['patterns'][i]['yardage'])
        yardage_max.append(pattern['patterns'][i]['yardage_max'])
        yarn_weight_description.append(pattern['patterns'][i]['yarn_weight_description'])
        generally_available.append(pattern['patterns'][i]['generally_available'])
        yardage_description.append(pattern['patterns'][i]['yardage_description'])
        photos_url.append(pattern['patterns'][i]['photos'][0]['square_url'])
        pattern_type_clothing.append(pattern['patterns'][i]['pattern_type']['clothing'])
        pattern_type_names.append(pattern['patterns'][i]['pattern_type']['permalink'])

    #     try:
        pattern_needle_sizes.append(pattern['patterns'][i]['pattern_needle_sizes'])
    #     except:
    #         pattern_needle_sizes.append(None)

        attributes = []
        try:
            for j in range(len(pattern['patterns'][i]['pattern_attributes'])):
                attributes.append(pattern['patterns'][i]['pattern_attributes'][j]['permalink'])
            pattern_attributes.append(attributes)
        except:
            pattern_attributes.append('None')

        try:
            category_dict = pattern['patterns'][i]['pattern_categories'][0]
            category_list = []
            category_list = [category_dict['permalink']]
            new_dict = category_dict['parent']
            while 'parent' in new_dict.keys():
                category_list.append(new_dict['permalink'])
                new_dict = new_dict['parent']
            categories.append(category_list)
        except:
            print("uhoh - check out categories!")
            
    # assemble dictionary          
    data = {'pattern_id':id_,
            'name':name,
            'name_permalink':name_permalink,
            'favorites_count': favorites_count,
            'projects_count': projects_count, 
            'difficulty_average' : difficulty_average, 
            'difficulty_count': difficulty_count, 
            'rating_average': rating_average,
            'queued_projects_count': queued_projects_count,
            'rating_count':rating_count,
            'pattern_type_names' :pattern_type_names,
            'pattern_type_clothing' :pattern_type_clothing,
            'photos_url' :photos_url,
            'pattern_needle_sizes' :pattern_needle_sizes,
            'pattern_attributes':pattern_attributes,
            'yardage_max' :yardage_max,
            'yardage' :yardage,
            'generally_available':generally_available,
            'gauge' :gauge,
            'gauge_divisor' :gauge_divisor,
            'free' :free,
            'downloadable': downloadable,
            'categories':categories,
            'yarn_weight_description' :yarn_weight_description,   
           }
    
    return pd.DataFrame(data)     

In [3]:
patterns_df.columns

NameError: name 'patterns_df' is not defined

In [4]:
def filter_and_save_csv(df):
    # remove any project with less than 5 projects
    
    # remove any with rating less than equal 3.5
    
    # do the actual saving
    df.to_csv('data/patterns_df.csv', mode ="a", index=False)
    


In [5]:
patterns_df = parse_json_response_into_df(response.json())
filter_and_save_csv(patterns_df)

NameError: name 'response' is not defined